In [5]:
import torch
import os
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_curve

In [6]:
from shutil import copy2

In [7]:
import matplotlib.pyplot as plt

In [8]:
races = ['African','Asian','Caucasian','Indian']
genders = ['Man','Woman']
labels = []
dist = []
for race in races:
    for gender in genders:
        with open(f"./sims/{race}_{gender}_sims.csv",'r') as f:
            lines = f.readlines()
        for line in lines:
            arr = line.strip().split(',')
            labels.append(int(arr[3]))
            dist.append(float(arr[2]))

In [9]:
fpr , tpr ,thresholds = roc_curve(labels,dist)
idx = np.argmax(tpr-fpr)
threshold = thresholds[idx]
print(f"Threshold: {threshold}")

Threshold: 0.5354980230331421


In [10]:
races = ['African','Asian','Caucasian','Indian']
genders = ['Man','Woman']
accuracies = {}
for race in races:
    for gender in genders:
        accuracies[f"{race}_{gender}"] = {'TP':0,'TN':0,'FP':0,'FN':0}
        with open(f"./sims/{race}_{gender}_sims.csv",'r') as f:
            lines = f.readlines()
        dist = []
        labels = []
        for line in lines:
            arr = line.strip().split(',')
            dist.append(float(arr[2]))
            labels.append(int(arr[3]))
            if float(arr[2]) < threshold:
                with open(f"./results/{race}_{gender}_results.csv",'a') as f:
                    f.write(f"{arr[0]},{arr[1]},0,{arr[3]}\n")
                if int(arr[3]) == 1:
                    accuracies[f"{race}_{gender}"]['TN'] += 1
                else:
                    accuracies[f"{race}_{gender}"]['FN'] += 1
            else:
                with open(f"./results/{race}_{gender}_results.csv",'a') as f:
                    f.write(f"{arr[0]},{arr[1]},1,{arr[3]}\n")
                if int(arr[3]) == 1:
                    accuracies[f"{race}_{gender}"]['TP'] += 1
                else:
                    accuracies[f"{race}_{gender}"]['FP'] += 1
        fpr, tpr, thresholds = roc_curve(labels,dist)
        fpr = np.array(fpr)
        tpr = np.array(tpr)
        with open(f"./roc/{race}_{gender}_roc.npy",'wb') as f:
            np.save(f,fpr)
            np.save(f,tpr)

In [11]:
print(accuracies)

{'African_Man': {'TP': 924, 'TN': 2015, 'FP': 900, 'FN': 2040}, 'African_Woman': {'TP': 26, 'TN': 35, 'FP': 15, 'FN': 45}, 'Asian_Man': {'TP': 941, 'TN': 1187, 'FP': 813, 'FN': 1318}, 'Asian_Woman': {'TP': 472, 'TN': 400, 'FP': 403, 'FN': 466}, 'Caucasian_Man': {'TP': 960, 'TN': 1293, 'FP': 834, 'FN': 1418}, 'Caucasian_Woman': {'TP': 474, 'TN': 273, 'FP': 290, 'FN': 458}, 'Indian_Man': {'TP': 1012, 'TN': 1303, 'FP': 844, 'FN': 1473}, 'Indian_Woman': {'TP': 443, 'TN': 242, 'FP': 396, 'FN': 286}}


In [12]:
gar = {}
far = {}
scores = {}
for race in races:
    for gender in genders:
        tp = accuracies[f"{race}_{gender}"]['TP']
        tn = accuracies[f"{race}_{gender}"]['TN']
        fp = accuracies[f"{race}_{gender}"]['FP']
        fn = accuracies[f"{race}_{gender}"]['FN']
        gar[f"{race}_{gender}"] = tp/(tn+tp)
        far[f"{race}_{gender}"] = fp/(fn+fp)
        scores[f"{race}_{gender}"] = 100*(fn+tp)/(fn+fp+tn+tp)

In [13]:
gar

{'African_Man': 0.31439265056141547,
 'African_Woman': 0.4262295081967213,
 'Asian_Man': 0.44219924812030076,
 'Asian_Woman': 0.5412844036697247,
 'Caucasian_Man': 0.42609853528628494,
 'Caucasian_Woman': 0.6345381526104418,
 'Indian_Man': 0.4371490280777538,
 'Indian_Woman': 0.6467153284671533}

In [14]:
far

{'African_Man': 0.30612244897959184,
 'African_Woman': 0.25,
 'Asian_Man': 0.38151102768653217,
 'Asian_Woman': 0.46375143843498273,
 'Caucasian_Man': 0.3703374777975133,
 'Caucasian_Woman': 0.3877005347593583,
 'Indian_Man': 0.3642641346568839,
 'Indian_Woman': 0.5806451612903226}

In [15]:
scores

{'African_Man': 50.416737540398024,
 'African_Woman': 58.67768595041322,
 'Asian_Man': 53.040619863817795,
 'Asian_Woman': 53.877082136703045,
 'Caucasian_Man': 52.7857935627081,
 'Caucasian_Woman': 62.34113712374582,
 'Indian_Man': 53.64853195164076,
 'Indian_Woman': 53.32845647403072}